In [78]:
import torch
import torch.nn as nn
import torch.nn.functional as F

先学习pytorch的官方文档，了解一下接下来要用到的函数

torch.nn.Conv2d
>torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')  
>Applies a 2D convolution over an input image composed of several input planes.  
>in_channels (int) – Number of channels in the input image  
>out_channels (int) – Number of channels produced by the convolution  
>kernel_size (int or tuple) – Size of the convolving kernel  
>stride (int or tuple, optional) – Stride of the convolution. Default: 1  
>padding (int or tuple, optional) – Zero-padding added to both sides of the input. Default: 0  
>padding_mode (string, optional) – zeros  
>dilation (int or tuple, optional) – Spacing between kernel elements. Default: 1  
>groups (int, optional) – Number of blocked connections from input channels to output channels. Default: 1  
>bias (bool, optional) – If True, adds a learnable bias to the output. Default: True  

torch.nn.Linear
>torch.nn.Linear(in_features, out_features, bias=True)  
>Applies a linear transformation to the incoming data: y = xA^T + b  
>in_features – size of each input sample  
>out_features – size of each output sample  
>bias – If set to False, the layer will not learn an additive bias. Default: True  

torch.nn.functional.relu
>torch.nn.functional.relu(input, inplace=False) → Tensor  
>Applies the rectified linear unit function element-wise. See ReLU for more details.  
>functionality: max(x,0)  
>inplace可选择是否替换原Tensor，默认为False不替换。  

torch.nn.functional.max_pool2d
>torch.nn.functional.max_pool2d(*args, **kwargs)  
>Applies a 2D max pooling over an input signal composed of several input planes.  


In [79]:
#define the network
#在Net类内，构造函数初始化了各个网络的结构，forward函数将这些网络串联起来，num_flat_features将image matrix铺平，打包成一个vector
#输入的image x(x1,x2,x3,x4), x1是samples数量，x2是初始channels数量，x3,x4是image大小
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3) #输入的in_channel是1，有6个kernel，且kernel_size=3，即filter的大小为(3,3)
        self.conv2 = nn.Conv2d(6, 16, 3) #输入的in_channel是6,即上一层的out_channel，有16个kernel，且kernel_size=3
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


查看一下各个网络层的参数数量。

In [80]:
params = list(net.parameters())
print(len(params))
for i in range(len(params)):
    print(params[i].size())

10
torch.Size([6, 1, 3, 3])
torch.Size([6])
torch.Size([16, 6, 3, 3])
torch.Size([16])
torch.Size([120, 576])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])


用一个随机32*32的image做输入，查看整个网络的输出结果。

In [81]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0077,  0.0483,  0.0722,  0.0104, -0.0451, -0.0692,  0.0956, -0.0860,
         -0.1316,  0.0710]], grad_fn=<AddmmBackward>)


loss function的计算

In [82]:
output=net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.4462, grad_fn=<MseLossBackward>)


反向传播，计算梯度

In [83]:
net.zero_grad()     # zeroes the gradient buffers of all parameters
loss.backward()
print(net.conv1.bias.grad)

tensor([ 0.0153, -0.0259, -0.0075, -0.0272,  0.0152,  0.0061])


更新权重，更新公式为：
weight = weight - learning_rate * gradient

In [84]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)
    
#或者用其它优化方法，以SGD举例
# import torch.optim as optim
# create your optimizer
# optimizer = optim.SGD(net.parameters(), lr=0.01)
# in your training loop:
# optimizer.zero_grad()   # zero the gradient buffers
# output = net(input)
# loss = criterion(output, target)
# loss.backward()
# optimizer.step()    # Does the update

output_test=net(input)
print(output_test)

tensor([[ 0.0417,  0.0545,  0.0772,  0.0236, -0.0493, -0.0634,  0.0856, -0.0976,
         -0.1348,  0.0834]], grad_fn=<AddmmBackward>)
